# Creación de modelos de puntuación del Riesgo de Impago

En esta aplicación implementaremos un modelo de puntuación de riesgo de impago siguiendo la metodología  que hemos analizado en la presentación de clase (selección de variables mediante valor de la información, tramificación de variables continuas, agrupación de categorías, transformación woe de variables, estimación de modelos de regresión logística, ....)

Existen diferentes librerías que incorporan funciones con los diferentes procedimientos ya programados que nos facilitan mucho la tarea. Una de estas librerías es `scorecardpy` [librería scorecardpy](https://pypi.org/project/scorecardpy/) que estima tarjetas de puntuación *lineales* utilizando regresiones logísticas. Esta librería nació inicialmente en R, y lamentablemente la versión de Python da algunos errores de adaptación a las últimas versiones de Pandas. Su desarrollador remite a utilizar la versión estable de R (librería en R 'scorecard'). 

Así que en su lugar de esta librería utilizaremos la librería `optBinning` [librería OptBinning](http://gnpalencia.org/optbinning/) que en realidad recoge (y en mi opinión mejora) la principal función de la librería `scorecardpy`  

## Creación de un entorno e instalación de librerías 



In [ ]:
# Podemos Crear un nuevo entorno de trabajo (mejor desde el terminal)
#! conda create -n risk_env
#! conda activate risk_env
#! conda config --env --add channels conda-forge
#! conda config --env --set channel_priority strict

# Instalamos la librería pandas y seaborn en el nuevo entrono desde conda (sólo la primera vez, utilizaré el terminal (con jupiter debo utilizar !))
# conda install pandas
# conda install seaborn

# Instalamos la librería optbinning y scorecardpy (sólo la primera vez)
# ! pip install scorecardpy # NOOO esta no la intaleis porque no es estable
# ! pip install optbinning


#Cargo o importo pandas, numpy, Matplotlib, 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para el análisis descriptivo inicial de contraste de asociación importo el test Chi2 y el anova 
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

# Cargaríamos la librería scortecardpy si fuese estable (que no lo es)
#import scorecardpy as sc


# Librería para hacer la tramificación, agrupación y transformación WOE
from optbinning import Scorecard, BinningProcess, OptimalBinning
from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks


# Scikit-learn para dividir la muestra y para estimar el modelo de regresión logística (sólo si no se quiere utilizar
# la función optbinning.scorecard que ya lo incropora)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

# Preparación de los datos sobre Créditos que utilizaremos en la práctica **germancredit**
## Carga de los datos 
En nuestra práctica utilizaremos una de las bases más utilizadas en los ejemplos de Puntuación de riesgo de crédito porque fue una de las primeras en ofrecerse en abierto. Corresponden a una base de datos de créditos de un Banco Alemán, y son datos reales, aunque muy antigüos de principios de los 90. Eso significa que las magnitudes de cantidades (expresadas en Marcos Alemanes), sean de difícil interpretación para el día de hoy. Sin embargo el signo y el sentido de las variables utilizadas para predecir el riesgo de impago de los futuros clientes permanece todavía de plena utilidad.

Los datos pueden descargarse del repositorio la UCI Machine Learning Repositoy [datos de Créditos alemanes](https://archive.ics.uci.edu/ml/datasets/Statlog+(German+Credit+Data)). Esta base de datos está incorporada en la librería `sorecardpy` pero mejor utilizamos el fichero original `germancredit.csv` que os he preparado yo


In [ ]:
# Cargamos los datos germancredit que están precargados en la librería scorecardpy
dt=pd.read_csv('germancredit.csv')

## Descripción inicial de los datos
Vamos a hacer una descripción inicial de los datos


In [ ]:
#Información del Contenido
dt.info()

## Variable objetivo: creditability

 La variable **creditability** es la *calidad crediticia* de  cada cliente, es la variable a predecir.    
 Toma originalmente dos valores (Buen Cliente y Mal Clioente)

In [ ]:
dt["creditability"].value_counts()


In [ ]:
#Recodifico esta variable creditability para que sea binaria y la llamo "y"

dt.rename(columns={"creditability":"y"},inplace=True)
dt['y'] = dt['y'].replace(['good', 'bad'], [0,1])


In [ ]:
dt["y"].value_counts()


In [ ]:
# También elimino la primera columna que no debería estar (es una variable de identificación que no debería tener en cuenta para la estimación del modelo)
dt.drop(labels='Unnamed: 0',inplace=True, axis=1)

dt.info()

## Variables Predictoras o explicativas

Las 20 restantes variables del data frame (7 numéricas y 13 categóricas) son los atributos o características observadas de esos clientes que se utilizarán para predecir la probabilidad de que los clientes cometan un impago de sis créditos, esto es, de que sean malos clientes. La descripción de estas 20 variables es la siguiente:

In [ ]:
#  - Attribute 1: (qualitative) **Status of existing checking account o cuenta corriente** 
#   - A11 : ... < 0 DM 
#   - A12 : 0 <= ... < 200 DM 
#   - A13 : ... >= 200 DM / salary assignments for at least 1 year 
#   - A14 : no checking account 

#- Attribute 2: (numerical) **Duration in month** 
  
#  - Attribute 3: (qualitative) **Credit history**  
#     - A30 : no credits taken/ all credits paid back duly (devultos sin mora)
#     - A31 : all credits at this bank paid back duly 
#     - A32 : existing credits paid back duly till now 
#     - A33 : delay in paying off in the past 
#     - A34 : critical account/ other credits existing (not at this bank) 

# - Attribute 4: (qualitative) **Purpose** 
#     - A40 : car (new) 
#     - A41 : car (used)
#     - A42 : furniture/equipment
#     - A43 : radio/television
#     - A44 : domestic appliances
#     - A45 : repairs
#     - A46 : education
#     - A47 : (vacation - does not exist?)
#     - A48 : retraining
#     - A49 : business
#    - A410 : others 

# - Attribute 5: (numerical) **Credit amount** 
  
# - Attribute 6: (qualitative) **Savings account/bonds** 
#    - A61 : ... < 100 DM
#    - A62 : 100 <= ... < 500 DM
#    - A63 : 500 <= ... < 1000 DM
#    - A64 : .. >= 1000 DM 
#    - A65 : unknown/ no savings account 

# - Attribute 7: (qualitative) **Present employment since** 
#     - A71 : unemployed 
#     - A72 : ... < 1 year 
#     - A73 : 1 <= ... < 4 years 
#     - A74 : 4 <= ... < 7 years 
#     - A75 : .. >= 7 years 

# - Attribute 8: (numerical) **Installment rate in percentage of disposable income** 
  
#  - Attribute 9: (qualitative) **Personal status and sex** 
#     - A91 : male : divorced/separated
#     - A92 : female : divorced/separated/married
#     - A93 : male : single
#     - A94 : male : married/widowed
#     - A95 : female : single 

# - Attribute 10: (qualitative) **Other debtors / guarantors**
#    - A101 : none
#    - A102 : co-applicant
#    - A103 : guarantor 

# - Attribute 11: (numerical) **Present residence since**
  
# - Attribute 12: (qualitative) **Property**
#     - A121 : real estate
#     - A122 : if not A121 : building society savings agreement/ life insurance
#     - A123 : if not A121/A122 : car or other, not in attribute 6 
#     - A124 : unknown / no property 

# - Attribute 13: (numerical) **Age in years** 
  
# - Attribute 14: (qualitative) **Other installment plans** Otros pagos por plazos
#      - A141 : bank
#      - A142 : stores
#      - A143 : none 

# - Attribute 15: (qualitative) **Housing**
#      - A151 : rent 
#      - A152 : own
#      - A153 : for free 

# - Attribute 16: (numerical) **Number of existing credits at this bank**
  
# - Attribute 17: (qualitative) **Job**
#   - A171 : unemployed/ unskilled - non-resident
#   - A172 : unskilled - resident
#   - A173 : skilled employee / official
#   - A174 : management/ self-employed/highly qualified employee/ officer 

# - Attribute 18: (numerical) **Number of people being liable to provide maintenance for** 
  
# - Attribute 19: (qualitative) **Telephone**
#   - A191 : none
#   - A192 : yes, registered under the customers name 

# - Attribute 20: (qualitative) **foreign worker** 
#   - A201 : yes 
#   - A202 : no 



## Analisis univariante y de asociación con la variable objetivo de las **variables categóricas**


### **status.of.existing.checking.account**

In [ ]:
dt['status.of.existing.checking.account'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(ctabla) 
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=1).round(3)

In [ ]:
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))


### **Personal status and sex**

In [ ]:
dt['personal.status.and.sex'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(ctabla) 
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))

### **Housing**

In [ ]:
dt['housing'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['housing'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
pd.crosstab(dt['housing'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(ctabla) 
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['housing'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))

### **Job**

In [ ]:
dt['job'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['job'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
pd.crosstab(dt['job'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(ctabla) 
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['job'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))

 ....  Habría que seguir haciendo esto con todas las variables categóricas para analizar asociación

## Analisis univariante y de asociación con la variable objetivo de las **Variables continuas**

In [ ]:
dt.describe()

### **Credit.amount**

In [ ]:
sns.displot(x=(dt['credit.amount']), kind="kde", fill=True)
plt.show()

In [ ]:
sns.displot(x=np.log(dt['credit.amount']), kind="kde", fill=True)
plt.show()

In [ ]:
dt['credit.amount']=np.log(dt['credit.amount'])

In [ ]:
sns.displot(x=(dt['credit.amount']), kind="kde", fill=True, hue=dt.y, common_norm=False)
plt.show()

In [ ]:
fvalue, pvalue = f_oneway(dt.loc[dt["y"]==0,['credit.amount']], dt.loc[dt["y"]==1,['credit.amount']])
print(fvalue, pvalue)


### **duration.in.month**

In [ ]:
sns.displot(x=(dt['duration.in.month']), kind="kde", fill=True)
plt.show()

In [ ]:
sns.displot(x=(dt['duration.in.month']), kind="kde", fill=True, hue=dt.y, common_norm=False)
plt.show()

In [ ]:
fvalue, pvalue = f_oneway(dt.loc[dt["y"]==0,['duration.in.month']], dt.loc[dt["y"]==1,['duration.in.month']])
print(fvalue, pvalue)

Tendría que continuar con el análisis del resto de variables continuas .....

In [ ]:
# sns.pairplot(dt, hue="y")

# **Selección de Variables**: análisis de Concentración para seleccionar las variables más **importantes** para meter en el modelo

### Dividimos la muestra en entrenamiento y test

In [ ]:
dt_train, dt_test = train_test_split(dt,stratify=dt["y"], test_size=.25, random_state=1234)

In [ ]:
dt_train["y"].shape

In [ ]:
dt_train["y"].mean()

In [ ]:
dt_test["y"].shape

In [ ]:
dt_test["y"].mean()

## Defino la tramificación óptima

## Tramificación de la Variable: "credit.amount"

In [ ]:
variable="credit.amount"
X=dt_train[variable].values
Y=dt_train['y'].values

In [ ]:
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

# Si se quisiese fijar los intervalos manualmente (porque no gusten los que obtine el agoritmo, entonces habría que usar:
#                     user_splits=
#                     user_splits_fixed=
# HAy veces que los datos tienen dátos missing y códigos especiales en este caso para obtener una categoría con esos datos missing y datos especiales hay que establecerlos
#                     special_codes = [-9, -8, -7]

# Una vez definido podemos pasar a estimarlo
optb.fit(X, Y)
optb.splits

Nota: Por defecto se utiliza un arbol de clasificación para hacer una tramificación inicial, y después se aplica un proceso de optimización de agrupación de categorías para maximizar el Valor de Información 

Una vez realizado el proceso de tramificación y agrupación óptima de categorías, obtenemos la tabla de agrupación 

In [ ]:
binning_table = optb.binning_table
binning_table.build()

Cabe mencionar que el WOE en esta tabla parece estar definido al revés que lo hemos hecho en clase, por lo que el signo es justo el contrario al que cabría esperar según lo que hemos visto en clase. En particular por defecto `optbinning` define el WOE de una categoría $i$ como
$$ WOE_i =  ln \left ( {Non-event_i \over Non-event_{total}} \over {Event_i \over Event_{total}}    \right ) $$

En este sentido los niveles con mayor tasa de impagados tendrán un WOE menos, y a medida que se reduzca la tasa de impagados (mejor calidad crediticia) irá aumentando el WOE. De hecho, 'optbinning' ni siquiera utiliza la misma fórmula que yo he utilizado en clase, por lo que no está acotada entre cero y uno, puede valos más que uno sin que eso signifique sobre ajuste.

Por talmotivo utilizaremos como criterio de selección exclusivamente IV<0.002

Podemos extraer el IV y el índice de Gini a partir de la tabla

In [ ]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))

# La última columna muestra el estadístico Jensen-Shannon de divergencia.
# Es una medida de la similaridad entre dos distribuciones de probabilidad (frecuencias de buenos y malos )
# que está acotada entre 0 y log2 (aprox 0.70) (puede utilizarse 0.01 como mínimo) 
print("JS= ", binning_table.js.round(3))



In [ ]:
# Podemos profundizar en el análisis estimando otras
binning_table.analysis(pvalue_test="chi2")
 

In [ ]:
# Por ejemplo otra medida que suele utilizarse en el Quality score(QS) que está acotada entre 0 y 1 (puede utilizarse 0.01 como mínimo)
print("QS= ", binning_table.quality_score.round(3))

In [ ]:
# La tabla anterior también muestra la V de Cramer (por encima de 0.20 podría ser suficiente para decir que hay asociación, 

# pero también podemos realizar el test con la tabla de contigencia completa:Jensen-Sha
x_transform_indices = optb.transform(X, metric="indices")

#pd.Series(x_transform_indices).value_counts(normalize=True,dropna=False).sort_index()
ctabla=pd.crosstab(pd.Series(x_transform_indices),Y,margins=True).round(3)
print(ctabla)

# Chi-square test of independence. Ho: Ausencia de Asociación (independencia)
c, p, dof, expected = chi2_contingency(ctabla) 
# Print the p-value
print("Test independencia. Estadístico :" ,round(c,3), "p-valor:", round(p,3))

Podemos realizar una representación gráfica de la Tabla de tramificación

In [ ]:
binning_table.plot(metric="woe")

In [ ]:
binning_table.plot(metric="event_rate")

Nótese que la relación entre lavariable x (credit.amount) y la tasa de evento (impago) es **no-lineal**

Ahora podemos aplicar esta tramificación óptima a la variable original y obtener la variable transformada WOE (que será una variable continua que utilizaremos en el modelode regresión)

In [ ]:
# Transformación WOE
x_woe = optb.transform(X, metric="woe")
pd.Series(x_woe).value_counts().sort_index()

Fíjate que ahora hemos conseguido "linealizar" la relación entre la variable trasnformada Woe y la propensión al impago 

In [ ]:
pd.crosstab(x_woe,Y,normalize=0).round(3)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pd.crosstab(x_woe,Y,normalize=0).iloc[:,1])
ax.set_xlabel("x_woe")
ax.set_ylabel("porcentaje de impago")

Nótese que para hacer la validación deberíamos hacer exactamente la misma transformación WOE, con la misma tramificación, al conjunto test. Para ello debemos aplicar la transformación optima calculada con el conjunto de entrenamiento, pero sobre la muestra de validación 

In [ ]:
# Transformación WOE en el conjunto test
x_test_woe = optb.transform(dt_test[variable].values, metric="woe")
pd.Series(x_test_woe).value_counts().sort_index()

Nótese que **no** estamos calculando una nueva tramificación para el conjunto de test, sino aplicando la tramificación obtenida con el conjunto de entrenamiento.    
En realidad si hiciéramos una tramificación óptima con el conjunto de test no tendría porqué salir igual que la estimada para el conunto de entrenamiento, como se puede comprobar a continuación

In [ ]:
variable="credit.amount"
X_test=dt_test[variable].values
Y_test=dt_test['y'].values
optb_test = OptimalBinning(name=variable, dtype="numerical")
optb_test.fit(X_test, Y_test)
print(optb_test.splits)
binning_table_test = optb_test.binning_table
binning_table_test.build()

Nótese que con el conjunto de test se han obtenido sólo 6 tramos y con diferentes puntos de corte ( y diferentes WOE), por eso es necesario no hacer una nueva tramificación al conjnto de test sino aplicar la tramificación obtenida usando en el conjunto de entrenamiento

## Tramificación de la duración en meses

In [ ]:
variable="duration.in.month"
X=dt_train[variable].values
Y=dt_train['y'].values

optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

optb.fit(X, Y)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))


In [ ]:
binning_table.plot(metric="woe")

In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
# Transformación WOE
x_woe = optb.transform(X, metric="woe")
pd.Series(x_woe).value_counts().sort_index

# Agrupción de niveles en variables Vbles Categóricas

En realidad, cuando tenemosvariables categóricas, no es necesario tramificar, pero sí hacer una agrupación de los diferentes niveles de forma que se maximice el *valor de información*

## Agrupación de la variable  *purpose*

In [ ]:
variable_cat = "purpose"
X_cat = dt_train[variable_cat].values
Y_cat = dt_train['y'].values

dt_train[variable_cat].value_counts()

In [ ]:

optb = OptimalBinning(name=variable_cat, dtype="categorical", solver="cp",
                      cat_cutoff=0.1)  # podemos cambiar los valores por defecto cat_cutoff=None, o, cat_cutoff=0.005

optb.fit(X_cat, Y_cat)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
x_woe = optb.transform(X_cat, metric="woe")
pd.Series(x_woe).value_counts()


__________________________________________________________________________________________________________________________________________________________________________________________________
__________________________________________________________________________________________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________________________________________________________________________________________
__________________________________________________________________________________________________________________________________________________________________________________________________

# Proceso de tramificación, agrupación y trasformación WOE Completo

Para no ir variable a variable se puede hacer todo el proceso completo

Proceso Entero



In [ ]:
# 1) Definimos la lista de nombres señalando cualse de ellas son las categóricas
Y = dt_train['y'].values
X = dt_train.drop(columns=['y']) #todas menos la primera que es el ID y la variable y
list_variables = X.columns.values.tolist()
list_categorical = X.select_dtypes(include=['object', 'category']).columns.values.tolist()

In [ ]:
# 2) Definimos el criterio de selección
selection_criteria = {
    "iv": {"min": 0.02}  # no imponemos "max": 1}
}

In [ ]:
# En caso de que desee modificarse los valores por defecto en el proceso de tramificación de alguna variable puede hacerse en forma de diccionario

binning_fit_params={
    "purpose":{"cat_cutoff": 0.10}
}

In [ ]:
# 3) Definimos el proceso de Tramificación o BinningProcess
binning_process = BinningProcess(
    categorical_variables=list_categorical,
    variable_names=list_variables,
    selection_criteria=selection_criteria,
    binning_fit_params=binning_fit_params)

In [ ]:
# 4) Obtenemos los tramos optimos de todas las Variables
dt_train_binned = binning_process.fit(X, Y)

In [ ]:
dt_train_binned.summary().sort_values('iv')

In [ ]:
# Ahora podemos ir sacando las tablas para cada variable

dt_train_binned.get_binned_variable("credit.amount").binning_table.build()

In [ ]:
dt_train_binned.get_binned_variable("purpose").binning_table.build()

In [ ]:
dt_train_binned.information()

In [ ]:
# las variables seleccionadas se pueden obtener con 'get_support'"Tarea Estudiantes_TarjetaPuntuacion"
dt_train_binned.get_support()

In [ ]:
# Podemos transformar las variables WOE
dt_train_woe=dt_train_binned.transform(X, metric="woe")


# Existe la posibilidad de obtener directamente las transformada si en lugar de usar fit, hubiésemos usado fit_transform 
# dt_train_binned = binning_process.fit_transform(X, Y)
# dt_train_binned.info()
# el resultado sería un data.frame con las X seleccionadas trasnsformadas WOE



In [ ]:
dt_train_woe.info()

In [ ]:
dt_train_woe

In [ ]:
# Ahora aplicaríamos la misma transformación pero al conjunto de test (si hubiera que puntuar a nuevos clientes haríamos lo mismo)

Y_test = dt_test['y'].values
X_test = dt_test.drop(columns=['y']) #todas menos la primera que es el ID y la variable y

dt_test_woe=dt_train_binned.transform(X_test, metric="woe")
dt_test_woe.info()





In [ ]:
dt_test_woe

# Estimación del Modelo

Ahora podemos calcular la tarjeta de puntuación. En los apuntes de clase definimos tanto los WOE, como los Odd ratio como la probabilidad de `evento` respecto al `no-evento` (malos clientes o impago=1 respecto a los buenos clientes o impago=0):

$$ odd = {{P}\over {(1-P)}} ~~ {,~~  siendo} ~~  {P=Prob(impago=1)} $$
 
 Y la fórmula para obtener la puntuación o los score debe ser una relación negativa con los odd ratio: cuanto mayor la probabilidad de impago (en relación a la de no impago), menor puntuación ha de tener: 

 $$ score {= offset - Factor}~·~{ln(odds)}$$

Para pasar de Probabiliddes de impago a Puntuaciones, habrá que establecer tanto el valor de `offset` como el de `Factor`. Esto se hace de manera arbitraria dependiendo de cada institución financiera.

En general, para determinar estos dos valores es necesario establecer la pendiente de la recta y un punto de la misma.

En cuanto a la pendiente, cuanto más plana sea la pendiente, menor variabilidad tendrán los valores de puntuación de crédito que se alcancen, y al revés, cuanto mayor pendiente más diferencias en la puntuación final. Yo voy a utilizar un apendiente (arbitraria) estableciendo de forma arbitraria cada cuantos puntos de score (**pdo_0**) se dobla el odd ratio: $ score - pdo_0 = {offset -Factor}~ ·{ln(2*odds)}$.

 En cuanto al punto de la recta (arbitrario), puede hacerse estableciento (de manera arbitraria) la puntuación o score considerada como de sobresaliente(**scorecard_points**) y el odd ratio que debería tener ese cliente de *sobresaliente* (**odds_0**)
 
 Así habría que establecer tres parámetros para transformar probabilidades de impago a puntuaciones, por ejemplo:   

* **pdo_0** =40  (esto es que cada 40 puntos de calidad creditica se dobla el odd-ratio))
* **scorecard_points** =600  (alguien con calidad crediticia muy buena, de sobresaliente, sacaría 600 puntos)
* **odds_0** =1/50  (odd ratio que se considera de sobresaliente)

La librería `optBinning` [librería OptBinning](http://gnpalencia.org/optbinning/), en realidad utiliza el módulo de `credit scoring` de `SAS-miner` como inspiración, y por eso define al revés tanto los WOE como los odd ratio, es decir `no-evento` en relación a `evento` (clientes buenos respecto a los malos, o no-impago respecto a impago, impago=0 respecto a impago=1). 
$$ odd^B = {{(1-P)}\over {P}} ~~ {,~~  siendo} ~~  {P=Prob(impago=1)} $$

Esto implica que la ecuación que transforma las probabilidades de impago en scores utilizando esta *odds<sup>B</sup>* debe tener pendiente positiva (cuanto mejor *odd<sup>B</sup>* mejor calidad crediticia tiene el cliente)

 $$ score= {offset + Factor} ~·~ {ln(odds^B)}$$
 
 Nótese que ahora habrá que establecer de nuevo los puntos de score que doblan el odd ratio (**pdo_0**), y también la puntuación o score considerada como de sobresaliente(**scorecard_points**) y el odd ratio que debería tener ese cliente de *sobresaliente* **odds_0 <sup>*B*</sup>**, con **odds_0 <sup>*B*</sup>** **= 1/odds_0**.

 Así para estimar la puntuación crediticia con `optBinning` hay que establecer tres parámetros para transformar probabilidades de impago a puntuaciones, por ejemplo:   

* **pdo_0** =40
* **scorecard_points** =600 
* **odds_0 <sup>*B*</sup>** = 50  (equivalente a **odds_0** =1/50 )




In [ ]:

# Directamente con el método Scorecard
estimator = LogisticRegression(solver="lbfgs")

# Establecemos los parámetros para la transformación de probabilidades en puntos de calidad crediticia o score

pdo_0 =40
scorecard_points_0= 600 
odds_0_B= 50 # (equivalente a  odds_0 =1/50 )

tarjeta= Scorecard(binning_process=binning_process,
                   estimator=estimator,
                   scaling_method="pdo_odds",
                   scaling_method_params={"pdo":pdo_0, "odds": odds_0_B, "scorecard_points": scorecard_points_0})

tarjeta.fit(X, Y, show_digits=4)

In [ ]:
tarjeta.information(print_level=2)

In [ ]:
tarjeta.table(style="detailed")

In [ ]:
# obtenmos las predicciones
Y_pred=tarjeta.predict_proba(X)[:,1]

# Calculamos la media
Y_pred.mean().round(5)


In [ ]:
# para contruir las matrices de confusión necesitamos determinar un
# **punto de corte de la probabilidad**.
# Ese punto de corte es el que me va a ayudar a realizar un pronóstico sobre los clientes:
# los malos clientes serán aquelloe para los que Prob Estimada > Prob_corte.


In [ ]:
# Para elegir el punto de corte puede utilizarse el Plot Kolmogorov-Smirnov (KS)
plot_ks(Y, Y_pred)

In [ ]:
# También puede utilizarse el máximo del f1_score
# definimos un vector de puntos de corte
c=np.arange(0,1,0.01)
# calculamos el f1_score para cada punto de corte
f1_score_ = [f1_score(dt_train["y"],np.multiply(Y_pred>c_,1)) for c_ in c]
# obtenemos el punto de corte que maximiza el f1_score
c_max = c[np.argmax(f1_score_)]
print("El punto de corte que maximiza el f1_score es: ", c_max)
print("y el máximo se alcanza en ", np.max(f1_score_).round(3))

# hacemos un gráfico de c y los f1_score correspondientes
plt.plot(c,f1_score_)
plt.stem(c_max, np.max(f1_score_),linefmt='r--', markerfmt='ro', basefmt='r--')
plt.title("Gráfico f1_score vs diferentes puntos de corte")
plt.text(c_max+0.05, 0, "c_max = "+str(c_max.round(2))+"\n f1_score_max = "+str(np.max(f1_score_).round(3)))
plt.show()


In [ ]:
# método de Youden (J) para obtener el punto de corte óptimo
# definimos un vector de puntos de corte (c) y de probabilidad de aceptación (p)
c=np.arange(0,1,0.01)

# Calculamos el estadístico J de Youden para cada punto de corte= Sensibilidad + Especificidad -1
J= [balanced_accuracy_score(dt_train["y"],np.multiply(Y_pred>c_,1), adjusted=True) for c_ in c ]
# obtenemos el punto de corte que maximiza el índice de Youden
c_max = c[np.argmax(J)]
print("El punto de corte que maximiza el índice de Youden es: ", c_max)
print("y el máximo se alcanza en ", np.max(J).round(3))

# gráfico de c y los índices de Youden correspondientes
plt.plot(c,J)
plt.stem(c_max, np.max(J),linefmt='r--', markerfmt='ro', basefmt='r--')
plt.title("Gráfico índice de Youden vs diferentes puntos de corte")
plt.text(c_max+0.05, 0, "c_max = "+str(c_max.round(2))+"\n J_max = "+str(np.max(J).round(3)))
plt.show()

In [ ]:
# Voy a utilizar simplemente la frecuencia observada
Prob_Corte=Y.mean()
print(Prob_Corte)

In [ ]:
dt_train["Y_pronostico"]=np.multiply(Y_pred>Prob_Corte,1)

In [ ]:
# Para comprobar la bondad de nuestras predicciones voy a comparar resultados con la tabla de confusión

# Primero estimo la precisión (los aciertos sobre el total de mis pronósticos)
pd.crosstab(dt_train["y"],dt_train["Y_pronostico"],margins=True, normalize=1).round(3)


Nótese que los falsos negativos (Bad Rate) es del 11.5% (préstamos aceptados o pronosticados como buenos que resultaron impagados)

In [ ]:
# Ahora estimo la exhaustividad o recall (Aciertos sobre los casos reales):
# la sensibilidad (sobre los positivos y=1), y la Especificidad (sobre los negativos y=0)

pd.crosstab(dt_train["y"],dt_train["Y_pronostico"],margins=True, normalize=0).round(3)

In [ ]:
# Por último un resumen global de aciertos
f1_score(dt_train["y"],dt_train["Y_pronostico"])

Para hacer la diagnosis también puedo utilizar medidas que no dependan crucialmente de un único punto de corte de Probabilidad

In [ ]:
# Diagnosis Curva ROC
plot_auc_roc(Y,Y_pred)

In [ ]:
# Diagnosis Cumulative Accuracy Profile (CAP)
# Otra curva alternativa a la curva ROC que permite evaluar la bondad de un modelo de clasificación es la curva CAP (Cumulative Accuracy Profile).
# La curva CAP se construye de la siguiente manera: ordenamos las observaciones de mayor a menor probabilidad
# de pertenecer a la clase positiva (y=1). A continuación, vamos acumulando las observaciones y calculando
# la proporción de positivos acumulados sobre el total de positivos. Esta proporción se representa en el eje Y.

# En el eje X representamos la proporción de observaciones acumuladas sobre el total de observaciones.
# La curva CAP se construye a partir de la curva de la línea recta (curva de referencia) y 
# la curva de la línea que representa la probabilidad estimada por el modelo.

plot_cap(Y, Y_pred)

In [ ]:
#### OJO que la diagnosis debe hacerse fuera de la muestra de entrenamiento
# obtenmos las predicciones
Y_test_pred=tarjeta.predict_proba(X_test)[:,1]

# Calculamos la media
Y_test_pred.mean().round(5)

In [ ]:
# Diagnosis Curva ROC
plot_auc_roc(Y_test,Y_test_pred)


In [ ]:
plot_cap(Y_test,Y_test_pred)

In [ ]:
dt_test["Y_pronostico"]=np.multiply(Y_test_pred>Prob_Corte,1)

# Primero estimo la precisión (los aciertos sobre el total de mis pronósticos)
print("\n Precisión:\n", pd.crosstab(dt_test["y"],dt_test["Y_pronostico"],margins=True, normalize=1).round(3))

# Ahora estimo la exhaustividad o recall (Aciertos sobre los casos reales):
# la sensibilidad (sobre los positivos y=1), y la Especificidad (sobre los negativos y=0)
print("\n exhaustividad:\n",pd.crosstab(dt_test["y"],dt_test["Y_pronostico"],margins=True, normalize=0).round(3))

# Por último un resumen global de aciertos
print("\n f1-score:",f1_score(dt_test["y"],dt_test["Y_pronostico"]))

In [ ]:
# Ahora vamos a calcular los score o puntuaciones.
#Que podemos hacer  con la función score
score = tarjeta.score(X)

print("Puntuación mínima: ", score.min().round(2))
print("Puntuación máxima: ",score.max().round(2))
print("Puntuación media : ",score.mean().round(2))  


# Transformación lineal según apuntes
# Factor= (pdo_0/log(2))
# Offset = scorecard_points_0+(pdo_0/log(2))*log(odds0_0)
# score= Offset - Factor *log(odds)

Factor= (pdo_0/np.log(2))
Offset = scorecard_points_0+Factor*np.log(1/odds_0_B)

#Podríamos haber calculado la puntuación también manualmente
score2= Offset-Factor*np.log(Y_pred/(1-Y_pred))

# Podemos comprobar que los resultados son los mismos   
print("Puntuación mínima: ",score2.min().round(2))
print("Puntuación máxima: ",score2.max().round(2))
print("Puntuación media : ",score2.mean().round(2))

datos_score=pd.DataFrame(np.transpose([score,score2, Y,Y_pred]), columns=['score','scoreManual','Y','Y_pred'])

# datos_score.to_excel("score_p1.xlsx")


Saber cómo se hace la transformación manual puede ayudarnos por ejemplo a la hora de establecer la `nota que determina el aprobado`. Imaginemos que utilizamos la frecuencia observada de impagos como probabilidad de corte 

In [ ]:

Score_Corte= Offset-Factor*np.log(Prob_Corte/(1-Prob_Corte))

print("La probabilida de corte de: ", Prob_Corte, " equivale a una puntuación de corte de: ", Score_Corte.round(2) )


In [ ]:
# Ahora representamos en un gráfico cómo separa el modelo a los buenos y los malos
datos_score=pd.DataFrame(np.transpose([score,Y]), columns=['score','Y'])
sns.displot(data=datos_score, x='score', label="event", hue='Y', alpha=0.35,kind="kde", fill=True, common_norm=True)
plt.axvline(Score_Corte, color='k', linestyle=":")

# Seguimiento del modelo: PSI (Population Stability Index)
El PSI es una medida de diferencia en la distribución de dos muestras, en nuestro caso entre la muestra utilizada para construir el modelo (entrenar y validar el modelo), y los nuevos datos que se vayan obteniendo con el transcurso del tiempo.  

Se aplica para detectar cuándo comienzan a verse diferencias entre las dos muestras (las puntuaciones de la muestra -train- y las puntuaciones obtenidas con los nuevos datos .... Cuando las distribuciones dejen de parecerse será el momento de revisar el modelo a tenor de los nuevos datos 

Como regla general 
  - **PSI <0.1**: No hay diferencias significativas entre las muestras de entrenamiento y los nuevos datos (resultado deseado, no se requiere más acciones)
  - **PSI entre 0.1 y 0.25** Hay cambio menores, valdría la pena revisar el modelo
  - **PSI >0.25** hay cambios importantes entre las dos muestras HAY QUE CAMBIAR EL MODELO 


In [ ]:
# Supongamos que tenemos un conjunto de nuevos datos que hemos ido recopilando después de la puesta en producción del modelo,
# y queremos utilizar esos nuevos datos para saber si es necesario revisar el modelo o si por el contrario podemos seguir utilizándolo

# Como en la base de datos no disponemos de este tipo de datos voy a suponer simplemente que los datos de test son los nuevos datos,

dt_nuevosdatos= dt_test.copy() 


from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks,ScorecardMonitoring


# Valores nuevos
Y_nuevo = dt_nuevosdatos['y'].values
X_nuevo = dt_nuevosdatos.drop(columns=['y']) #todas menos la primera que es el ID y la variable y

# Valores de entrenamiento
Y = dt_train['y'].values
X = dt_train.drop(columns=['y']) #todas menos la variable y


# ¿se distibuyen igual las probabilidades esperadas?
score_train = tarjeta.score(X)
score_nuevo = tarjeta.score(X_nuevo)

datos_score_psi1=pd.DataFrame(np.transpose([score_train,Y]), columns=['score','Y'])
datos_score_psi1['tipo']='entrenamiento'

datos_score_psi2=pd.DataFrame(np.transpose([score_nuevo,Y_nuevo]), columns=['score','Y'])
datos_score_psi2['tipo']='nuevos datos'

datos_score_psi= pd.concat([datos_score_psi1,datos_score_psi2])
sns.displot(data=datos_score_psi, x='score', label="event", hue='tipo', alpha=0.35,kind="kde", fill=True,common_norm=False)
plt.axvline(Score_Corte, color='k', linestyle=":")




# Estimo el psi
# Defino la tarjeta a evalear
psi=ScorecardMonitoring(tarjeta, psi_method="cart",psi_min_bin_size=0.10, psi_n_bins=20)
#psi=ScorecardMonitoring(tarjeta, psi_method= "quantile", psi_n_bins=10)


psi.fit(X_actual=X_nuevo, y_actual=Y_nuevo, X_expected=X, y_expected=Y)

psi.psi_plot()
psi.psi_table()
psi.tests_table()


In [ ]:
psi.system_stability_report()

In [ ]:
psi.psi_variable_table(style="summary").sort_values('PSI')